# The analysis of multiscale model results & running TDSE interactively

This notebook shows various analyses of the results of the multiscale model. First, we investigate the pulse shaping due to the non-linear propagation, the plasma channel. Then we analyse the build-up of the XUV signal and its structure. Next, we will get more insight directly into the generating process inside the medium by studying the spectra. To conclude the analysis, we outline possibilities to fasten the analysis. Finally, we show the TDSE solver applied to a custom electric field defined directly in this notebook. We use various tools to analyse the result.

## Load libraries & initial data

In [ ]:
## python modules used within this notebook
import numpy as np
from scipy import integrate
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.colors as colors
import os
import h5py
import sys
import MMA_administration as MMA
import mynumerics as mn
import units
from IPython.display import display, Markdown
from IPython.display import HTML

import dataformat_CUPRAD as dfC
import HHG
import plot_presets as pp 

matplotlib.rcParams['animation.embed_limit'] = 200.
# print(matplotlib.rcParams['animation.embed_limit'])


# %%capture
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()

# %matplotlib agg
%matplotlib inline

## Load data

We load the data from the pulse propagation in the Pythonic data container. It contains the data about the pulse propagation and some firther characteristics. The data from the micrscopic response and harmonic signal will be loaded later (these data are large, and we will need only a part of them according to the chosen analyses.)

In [ ]:
# series_version = 'v3'
# h5file1 = os.path.join('/mnt','d','sharepoint', 'OneDrive - ELI Beamlines', 'data', 'Sunrise', 'demos', 'Bessel', series_version, 'results_Bessel_7.h5')
# h5file2 = os.path.join('/mnt','d','sharepoint', 'OneDrive - ELI Beamlines', 'data', 'Sunrise', 'demos', 'Bessel', series_version, 'results_Bessel_9.h5')

demos_path = os.path.join(os.environ['MULTISCALE_DEMOS'],'Bessel')
series_version = 'v4'
h5file1 = os.path.join(demos_path, series_version, 'results_Bessel_1.h5')
h5file2 = os.path.join(demos_path, series_version, 'results_Bessel_7.h5')



# simulation = 'bigpf'
# h5file1 = os.path.join('/mnt','d', 'data', 'Sunrise', simulation, 'results.h5')
# h5file2 = os.path.join('/mnt','d', 'data', 'Sunrise', simulation, 'results_Hankel.h5')

with h5py.File(h5file1,'r') as f, h5py.File(h5file2,'r') as f2:

    # load cuprad data = pulse propagation
    CUPRAD_res = dfC.get_data(f)
    CUPRAD_res2 = dfC.get_data(f2)
    # CUPRAD_res.get_plasma(f)


## Plot the propagating pulse
We choose the time-and-space window to see the pulse as it propagates through the medium. Note that we measure the intensity by the "expected harmonic cutoff", these units are obtained by the formula $E_{\text{cut-off}} = I_P + 3.17U_p$ (it is directly proportional since $U_p$ is linearly proportional to the intensity). Then we plot the plasma channel create by the passage of the pulse. We show both absolute density of free electrons and also relative to the local density.

There are more technical details about the data: We plot the pulse directly as it is stored in the file. This means that we a co-moving frame defined by the group velocity, $v_g$, of the pulse: this is the computational window of CUPRAD. The group velocity $v_g$ is defined from the linear dispersion relation and depends on the chosen reference pressure and central wavelength. Possible density modulation is relative to this reference pressure, whcih is the reason we use the average pressure in our examples. Physically speaking, $v_g$ is arbitrary and needs to be considered in further processing. For example, the Pyrhonic class represented by `CUPRAD_res` contains methods to adjust to the reference given by the speed of light (both activelly by changing the data or just by sychronising the clocks in the $t$-grid).

In [ ]:

tlim = np.asarray([t_plot_span*CUPRAD_res.pulse_duration_entry for t_plot_span in (-2. , 2.)])
tlim2 = np.asarray([t_plot_span*CUPRAD_res2.pulse_duration_entry for t_plot_span in (-2. , 2.)])

# ani_outpath = os.path.join('/mnt','d', 'git', 'MMA-interactive', 'export')
ani_outpath = os.path.join(os.environ['MULTISCALE_WORK_DIR'],'Bessel', 'export')
if not(os.path.exists(ani_outpath)): os.makedirs(ani_outpath)
# rlim = 600

In [ ]:
# Code to generate the animated figure

k_t_min, k_t_max = mn.FindInterval(1e15*CUPRAD_res.tgrid,1.05*tlim)
k_t_min2, k_t_max2 = mn.FindInterval(1e15*CUPRAD_res2.tgrid,1.05*tlim2)

# k_r_max          = mn.FindInterval(1e6*CUPRAD_res.rgrid ,1.05*rlim)

fig = plt.figure(figsize=(14, 5.5))


ax1 = plt.subplot2grid((1, 2), (0, 0))  # Upper left
ax2 = plt.subplot2grid((1, 2), (0, 1))  # Upper right

# r_grid, sym_data = mn.symmetrize_y(1e6*CUPRAD_res.rgrid[:k_r_max],
#                     (
#                     HHG.ComputeCutoff(
#                         mn.FieldToIntensitySI(CUPRAD_res.E_zrt[0,:k_r_max,k_t_min:k_t_max])/units.INTENSITYau,
#                         mn.ConvertPhoton(CUPRAD_res.omega0,'omegaSI','omegaau'),
#                         mn.ConvertPhoton(CUPRAD_res.Ip_eV,'eV','omegaau')
#                     )[1]
#                     ).T)

pc1 = ax1.pcolormesh(1e15*CUPRAD_res.tgrid[k_t_min:k_t_max], 1e6*CUPRAD_res.rgrid, CUPRAD_res.E_zrt[0,:,k_t_min:k_t_max], shading='auto', cmap = 'seismic')
pc2 = ax2.pcolormesh(1e15*CUPRAD_res2.tgrid[k_t_min2:k_t_max2], 1e6*CUPRAD_res2.rgrid, CUPRAD_res2.E_zrt[0,:,k_t_min2:k_t_max2], shading='auto', cmap = 'seismic')

ax1.set_xlim(tlim)
# ax1.set_ylim((-rlim,rlim))

ax1.set_title("z={:.2f}".format(1e3*CUPRAD_res.zgrid[0]) + ' mm')
ax1.set_xlabel(r'$t~[\mathrm{fs}]$')
ax1.set_ylabel(r'$\rho~[\mu\mathrm{m}]$')

cbar1 = fig.colorbar(pc1, ax=ax1)
cbar2 = fig.colorbar(pc2, ax=ax2)
# cbar.ax.set_ylabel(r'Intensity [harmonic cut-off]', rotation=90)




def update(frame):
    # Update the data
    data1 = CUPRAD_res.E_zrt[frame,:,k_t_min:k_t_max]
    data2 = CUPRAD_res2.E_zrt[frame,:,k_t_min2:k_t_max2]
    
    # Update the colors
    pc1.set_array(data1.ravel())
    pc1.set_clim(data1.min(), data1.max())
    cbar1.update_normal(pc1)

    pc2.set_array(data2.ravel())
    pc2.set_clim(data2.min(), data2.max())
    cbar2.update_normal(pc2)

    ax1.set_title("z={:.2f}".format(1e3*CUPRAD_res.zgrid[frame]) + ' mm')

    # Update the progress indicator
    # progress_line.set_data([1e3*CUPRAD_res.zgrid[frame], 1e3*CUPRAD_res.zgrid[frame]],
    #                         [CUPRAD_res.density_mod_profile_mbar.min(), CUPRAD_res.density_mod_profile_mbar.max()])

    return [pc1,pc2]

# Ensure the layout does not have overlaps and everything is nicely spaced
fig.tight_layout() 

ani = matplotlib.animation.FuncAnimation(fig, update, frames=len(CUPRAD_res.zgrid), blit=True)



# Define the writer using ffmpeg for mp4 format and save it
Writer = matplotlib.animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Jan Vabek'), bitrate=1800)

ani.save(os.path.join(ani_outpath,'Bessel_pulse.mp4'), writer=writer)

HTML(ani.to_jshtml())
